In [ ]:
Validation code

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import tensorflow as tf
import random as python_random
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
from itertools import product

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
python_random.seed(42)

# Load datasets
rainfall_data = pd.read_excel('AggregatedDailyRainfall.xlsx')
coordinates_data = pd.read_excel('LATLONRainGaugeAWS.xlsx')
elevation_data = pd.read_excel('GaugesElevation.xlsx')

# Preprocessing Data
rainfall_columns = rainfall_data.columns[1:]  # this selects all columns except the first
rolling_means = rainfall_data[rainfall_columns].rolling(window=60, min_periods=1).mean()
rainfall_data = pd.concat([rainfall_data.iloc[:, 0], rolling_means], axis=1)

# Merge DataFrames
data = pd.merge(rainfall_data.melt(id_vars='Date', var_name='Location', value_name='Rainfall'), coordinates_data, on='Location', how='left')
data = pd.merge(data, elevation_data, on='Location', how='left')
data['Date'] = pd.to_datetime(data['Date'])

# Extract year, month, and day from the date
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Day'] = data['Date'].dt.day

# Normalize numerical features except 'Year'
scaler = MinMaxScaler()
data[['Latitude', 'Longitude', 'Elevation', 'Rainfall']] = scaler.fit_transform(data[['Latitude', 'Longitude', 'Elevation', 'Rainfall']])
scaler_rainfall = scaler

# Cyclic encoding for Month and Day
data['Month_sin'] = np.sin(2 * np.pi * data['Month'] / 12)
data['Month_cos'] = np.cos(2 * np.pi * data['Month'] / 12)
data['Day_sin'] = np.sin(2 * np.pi * data['Day'] / 31)
data['Day_cos'] = np.cos(2 * np.pi * data['Day'] / 31)

# Option 1: Scaling Year
data_scaled = data.copy()
data_scaled['Year'] = scaler.fit_transform(data[['Year']])

# Option 2: One-Hot Encoding Year
#encoder = OneHotEncoder(sparse_output=False)
#encoded_year = encoder.fit_transform(data[['Year']])
#encoded_year_df = pd.DataFrame(encoded_year, columns=encoder.get_feature_names_out(['Year']))
#data_onehot = pd.concat([data.reset_index(drop=True), encoded_year_df], axis=1)

# Prepare the input and output data for the model for both options
X_scaled = data_scaled[['Latitude', 'Longitude', 'Elevation', 'Year', 'Month_sin', 'Month_cos', 'Day_sin', 'Day_cos']]
#X_onehot = data_onehot[['Latitude', 'Longitude', 'Elevation', 'Month_sin', 'Month_cos', 'Day_sin', 'Day_cos'] + list(encoded_year_df.columns)]
y = data['Rainfall']

# Function to unscale the data
def unscale(data, scaler):
    return scaler.inverse_transform(data)

# Function to run the model with given input features and hyperparameters
def run_model(X, y, scaler_rainfall, plot_filename, layers, units, epochs, batch_size):
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    metrics_results = {'mae_train': [], 'mse_train': [], 'rmse_train': [], 'r2_train': [], 'correlation_train': [],
                       'mae_test': [], 'mse_test': [], 'rmse_test': [], 'r2_test': [], 'correlation_test': []}
    all_train_loss = []
    all_val_loss = []

    for fold, (train_index, test_index) in enumerate(kf.split(X)):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Build the model
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Input(shape=(X_train.shape[1],)))  # Explicit Input layer
        for _ in range(layers):
            model.add(tf.keras.layers.Dense(units, activation='relu'))
        model.add(tf.keras.layers.Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

        history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=0)
        all_train_loss.append(history.history['loss'])
        all_val_loss.append(history.history['val_loss'])

        predictions_train = model.predict(X_train)
        predictions_test = model.predict(X_test)

        # Unscale the predictions and the target values
        predictions_train_unscaled = unscale(predictions_train, scaler_rainfall)
        predictions_test_unscaled = unscale(predictions_test, scaler_rainfall)
        y_train_unscaled = unscale(y_train.values.reshape(-1, 1), scaler_rainfall)
        y_test_unscaled = unscale(y_test.values.reshape(-1, 1), scaler_rainfall)

        metrics_results['mae_train'].append(mean_absolute_error(y_train_unscaled, predictions_train_unscaled))
        metrics_results['mse_train'].append(mean_squared_error(y_train_unscaled, predictions_train_unscaled))
        metrics_results['rmse_train'].append(np.sqrt(metrics_results['mse_train'][-1]))
        metrics_results['r2_train'].append(r2_score(y_train_unscaled, predictions_train_unscaled))
        metrics_results['correlation_train'].append(np.corrcoef(y_train_unscaled.flatten(), predictions_train_unscaled.flatten())[0, 1])
        
        metrics_results['mae_test'].append(mean_absolute_error(y_test_unscaled, predictions_test_unscaled))
        metrics_results['mse_test'].append(mean_squared_error(y_test_unscaled, predictions_test_unscaled))
        metrics_results['rmse_test'].append(np.sqrt(metrics_results['mse_test'][-1]))
        metrics_results['r2_test'].append(r2_score(y_test_unscaled, predictions_test_unscaled))
        metrics_results['correlation_test'].append(np.corrcoef(y_test_unscaled.flatten(), predictions_test_unscaled.flatten())[0, 1])
        
    # Calculate average loss across folds
    avg_train_loss = np.mean(np.array(all_train_loss), axis=0)
    avg_val_loss = np.mean(np.array(all_val_loss), axis=0)

    plt.figure(figsize=(12, 8))
    plt.plot(avg_train_loss, label='Average Train Loss')
    plt.plot(avg_val_loss, label='Average Validation Loss', linestyle='--')
    plt.title('Average Model Loss by Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(plot_filename)
    plt.show()

    for key in metrics_results.keys():
        average_metric = np.mean(metrics_results[key])
        print(f"Average {key}: {average_metric:.3f}")

    return metrics_results

# Define hyperparameters to test
layer_options = [2]  # Number of hidden layers
unit_options = [50]  # Number of units per layer
epoch_options = [50]  # Number of epochs
batch_size_options = [32]  # Batch size

# Running and comparing models with different hyperparameters
for layers, units, epochs, batch_size in product(layer_options, unit_options, epoch_options, batch_size_options):
    plot_filename = f'average_model_loss_scaled_layers{layers}_units{units}_epochs{epochs}_batch{batch_size}.png'
    print(f"\nRunning model with {layers} layers, {units} units, {epochs} epochs, {batch_size} batch size")
    metrics_scaled = run_model(X_scaled, y, scaler_rainfall, plot_filename, layers, units, epochs, batch_size)
    
    #plot_filename = f'average_model_loss_onehot_layers{layers}_units{units}_epochs{epochs}_batch{batch_size}.png'
    #print(f"\nRunning model with {layers} layers, {units} units, {epochs} epochs, {batch_size} batch size (One-Hot Encoded Year)")
    #metrics_onehot = run_model(X_onehot, y, scaler_rainfall, plot_filename, layers, units, epochs, batch_size)


In [ ]:
production code

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import tensorflow as tf
import random as python_random
#import matplotlib.pyplot as plt
from keras.models import Sequential
import sklearn  # Import scikit-learn

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
python_random.seed(42)

# Load datasets
rainfall_data = pd.read_excel('AggregatedDailyRainfall.xlsx')
coordinates_data = pd.read_excel('LATLONRainGaugeAWS.xlsx')
elevation_data = pd.read_excel('GaugesElevation.xlsx')
new_locations = pd.read_excel('DEM4Elevation_5431pts.xlsx')

# Preprocessing Data
rainfall_columns = rainfall_data.columns[1:]  # this selects all columns except the first
rolling_means = rainfall_data[rainfall_columns].rolling(window=60, min_periods=1).mean()
rainfall_data = pd.concat([rainfall_data.iloc[:, 0], rolling_means], axis=1)

# Merge DataFrames
data = pd.merge(rainfall_data.melt(id_vars='Date', var_name='Location', value_name='Rainfall'), coordinates_data, on='Location', how='left')
data = pd.merge(data, elevation_data, on='Location', how='left')
data['Date'] = pd.to_datetime(data['Date'])

# Extract year, month, and day from the date
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Day'] = data['Date'].dt.day

# Normalize numerical features
scaler_long_lat_elev = MinMaxScaler()
data[['Latitude', 'Longitude', 'Elevation']] = scaler_long_lat_elev.fit_transform(data[['Latitude', 'Longitude', 'Elevation']])

scaler_rainfall = MinMaxScaler()
data['Rainfall'] = scaler_rainfall.fit_transform(data[['Rainfall']])

# Cyclic encoding for Month and Day
data['Month_sin'] = np.sin(2 * np.pi * data['Month'] / 12)
data['Month_cos'] = np.cos(2 * np.pi * data['Month'] / 12)
data['Day_sin'] = np.sin(2 * np.pi * data['Day'] / 31)
data['Day_cos'] = np.cos(2 * np.pi * data['Day'] / 31)

# Scaling Year
data_scaled = data.copy()
scaler_year = MinMaxScaler()
data_scaled['Year'] = scaler_year.fit_transform(data[['Year']])

# Prepare the input and output data for the model
X_scaled = data_scaled[['Latitude', 'Longitude', 'Elevation', 'Year', 'Month_sin', 'Month_cos', 'Day_sin', 'Day_cos']]
y = data['Rainfall']

# Function to unscale the data
def unscale(data, scaler):
    return scaler.inverse_transform(data)

# Function to train the final model with given input features and hyperparameters
def train_final_model(X, y, layers, units, epochs, batch_size):
    # Build the model
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(X.shape[1],)))  # Explicit Input layer
    for _ in range(layers):
        model.add(tf.keras.layers.Dense(units, activation='relu'))
    model.add(tf.keras.layers.Dense(1)) 
    
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

    # Train the model on the full dataset without validation split
    model.fit(X, y, epochs=epochs, batch_size=batch_size, verbose=0)

    return model

# Training the final model with selected hyperparameters
layers = 2
units = 50
epochs = 50
batch_size = 32

# Train on scaled year data
print("Training final model with scaled year data...")
final_model_scaled = train_final_model(X_scaled, y, layers, units, epochs, batch_size)


# Prepare new location data for predictions for all unique dates
unique_dates = data['Date'].unique()
predictions_list = []

for current_date in unique_dates:
    new_locations['Date'] = current_date
    #new_locations['Date'] = pd.to_datetime(new_locations['Date'])
    new_locations['Year'] = new_locations['Date'].dt.year
    new_locations['Month'] = new_locations['Date'].dt.month
    new_locations['Day'] = new_locations['Date'].dt.day

    new_locations['Month_sin'] = np.sin(2 * np.pi * new_locations['Month'] / 12)
    new_locations['Month_cos'] = np.cos(2 * np.pi * new_locations['Month'] / 12)
    new_locations['Day_sin'] = np.sin(2 * np.pi * new_locations['Day'] / 31)
    new_locations['Day_cos'] = np.cos(2 * np.pi * new_locations['Day'] / 31)

    # Normalize new location features
    new_locations[['Latitude', 'Longitude', 'Elevation']] = scaler_long_lat_elev.transform(new_locations[['Latitude', 'Longitude', 'Elevation']])
    new_locations['Year'] = scaler_year.transform(new_locations[['Year']])

    # Prepare new location input data
    new_X_scaled = new_locations[['Latitude', 'Longitude', 'Elevation', 'Year', 'Month_sin', 'Month_cos', 'Day_sin', 'Day_cos']]

    # Predict rainfall at new locations
    predictions_scaled = final_model_scaled.predict(new_X_scaled, verbose =0)
    predictions_scaled_unscaled = unscale(predictions_scaled, scaler_rainfall)

    # Add predictions to new locations dataframe
    new_locations['Predicted_Rainfall'] = predictions_scaled_unscaled

    # Unscale Latitude, Longitude, and Elevation to original scale
    new_locations[['Latitude', 'Longitude', 'Elevation']] = scaler_long_lat_elev.inverse_transform(new_locations[['Latitude', 'Longitude', 'Elevation']])

    # Save predictions for the current date
    predictions_list.append(new_locations.copy())

# Combine predictions for all dates
all_predictions = pd.concat(predictions_list)

# Save the predictions to a new Excel file in the specified format
results = all_predictions.pivot(index=['Longitude', 'Latitude', 'Elevation'], columns='Date', values='Predicted_Rainfall')
results.reset_index(inplace=True)
results.to_excel('Predicted_Rainfall_New_Locations_All_Dates_Organized.xlsx', index=False)

print("Predictions saved to 'Predicted_Rainfall_New_Locations_All_Dates.xlsx'")